<a href="https://colab.research.google.com/github/tlysenko/structure-aware-protein-solubility-prediction/blob/main/2_D_features_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Installing packages

In [ ]:
!pip install Bio

### Importing the libraries

In [1]:
import os

import pandas as pd
import numpy as np
from Bio import SeqIO
import os
import io
import requests as r

%tensorflow_version 1.x
import tensorflow  
import tqdm
import pickle
import stat

TensorFlow 1.x selected.


### File Downloads

In [60]:
# Copy github repo of the hachathon
#os.chdir('/content/drive/MyDrive/Colab Notebooks/Solubility-tool')
#!git clone https://github.com/ProteinQure/cbh21-protein-solubility-challenge

In [ ]:
# SPOT-Contact folder
#!wget https://servers.sparks-lab.org/downloads/SPOT-Contact_local.tgz
#!tar -xvf 'SPOT-Contact_local.tgz' -C '/content/drive/MyDrive/Colab Notebooks/Solubility-tool'

#wget https://servers.sparks-lab.org/downloads/SPIDER3-Single_np.tgz
#!tar -xvf 'SPIDER3-Single_np.tgz' -C '/content/drive/MyDrive/Colab Notebooks/Solubility-tool'

In [ ]:
#PSIBLAST 
#https://ftp.ncbi.nlm.nih.gov/blast/executables/blast+/LATEST/
#https://www.ncbi.nlm.nih.gov/books/NBK52640/
#!wget https://ftp.ncbi.nlm.nih.gov/blast/executables/blast+/LATEST/ncbi-blast-2.11.0+-x64-linux.tar.gz
#!tar -xvf 'ncbi-blast-2.11.0+-src.tar.gz' -C '/content/drive/MyDrive/ColabNotebooks/Solubility-tool'

### PDB to FASTA

In [24]:
root = '/content/drive/MyDrive/ColabNotebooks/Solubility-tool/cbh21-protein-solubility-challenge/data/'

In [25]:
def get_fasta_pdb(path):
  ids = []
  seqs = []

  for (dirpath, dirnames, filenames) in os.walk(path):
      for filename in filenames:
          if filename.split(".")[1] == "pdb":
              ids.append(filename.split(".")[0])
              seq_ = SeqIO.parse(dirpath + filename, 'pdb-seqres')
              for i in seq_:
                  seq = str(i.seq)
                  if seq:
                    seqs.append(seq)
                    break

  if not seqs:
        seqs = get_fasta_up(ids)

  return ids, seqs

def get_fasta_up(list_): # list of file ids
  baseUrl="http://www.uniprot.org/uniprot/"
  seqs = []
  for cID in list_:
    currentUrl=baseUrl+cID+".fasta"
    response = r.post(currentUrl)
    cData=''.join(response.text)
    Seq=io.StringIO(cData)
    pSeq=list(SeqIO.parse(Seq,'fasta'))
    try:
      seqs.append(str(pSeq[0].seq))
    except:
      seqs.append(None)
  return seqs

In [26]:
ecid, ecseq = get_fasta_pdb(root + "/training/crystal_structs/")
ycid, ycseq = get_fasta_pdb(root + "/test/yeast_crystal_structs/")

In [27]:
emid, emseq = get_fasta_pdb(root + "/test/ecoli_modelled_structs/")
ymid, ymseq = get_fasta_pdb(root + "/test/yeast_modelled_structs/")

In [28]:
print(len(ecid), len(ecseq))
print(len(emid), len(emseq))
print(len(ycid), len(ycseq))
print(len(ymid), len(ymseq))


406 406
550 550
59 59
50 50


### Make Fasta files

In [29]:
train_id = ecid
train_seq = ecseq

test_id = emid + ycid + ymid
test_seq = emseq + ycseq + ymseq

df_train = pd.DataFrame()
df_train["id"] = train_id
df_train["seq"] = train_seq

df_test = pd.DataFrame()
df_test["id"] = test_id
df_test["seq"] = test_seq

In [34]:
root = '/content/drive/MyDrive/ColabNotebooks/Solubility-tool/cbh21-protein-solubility-challenge/data/'
for i in range(df_train.shape[0]):
  id = ">pdb:" + df_train.loc[i, "id"]
  seq = df_train.loc[i, "seq"]
  with open(root + f'fasta_train/{df_train.loc[i, "id"]}.fasta', 'w') as file:
    file.write(f"{id}\n{seq}")

In [35]:
for i in range(df_test.shape[0]):
  id = ">pdb:" + df_test.loc[i, "id"]
  seq = df_test.loc[i, "seq"]
  with open(root + f'fasta_test/{df_test.loc[i, "id"]}.fasta', 'w') as file:
    file.write(f"{id}\n{seq}")

### Test run SPOT-Contact 

In [26]:
def change_permissions_recursive(path, mode):
    for root, dirs, files in os.walk(path, topdown=False):
        for dir in [os.path.join(root,d) for d in dirs]:
            os.chmod(dir, mode)
    for file in [os.path.join(root, f) for f in files]:
            os.chmod(file, mode)

fff = '/content/drive/MyDrive/ColabNotebooks/Solubility-tool/SPOT-Contact-Helical-New/sources/CCMpred' 
change_permissions_recursive(fff, 0o777)

In [24]:
file = '/content/drive/MyDrive/ColabNotebooks/Solubility-tool/SPOT-Contact-Helical-New/sources/CCMpred/calCCMpred.sh'
os.chmod(file, 0o777)

In [27]:
# Go, go baby!
os.chdir('/content/drive/MyDrive/ColabNotebooks/Solubility-tool/SPOT-Contact-Helical-New/')

!sh 'run_spotcontacthelical.sh'

Protein: tmp
Generating PSSM...
uuuuiiii: /content/drive/MyDrive/ColabNotebooks/Solubility-tool/ncbi-blast-2.11.0+
Generating HHblits outputs...
Generating SPIDER3 outputs...
Generating CCMPRED outputs...
Generating DCA outputs...
Protein: P00445
Generating PSSM...
uuuuiiii: /content/drive/MyDrive/ColabNotebooks/Solubility-tool/ncbi-blast-2.11.0+
run_spotcontacthelical.sh: 32: run_spotcontacthelical.sh: /content/drive/MyDrive/ColabNotebooks/Solubility-tool/ncbi-blast-2.11.0+: Permission denied
Generating HHblits outputs...
run_spotcontacthelical.sh: 34: run_spotcontacthelical.sh: hhblits: not found
Generating SPIDER3 outputs...
run_spotcontacthelical.sh: 37: run_spotcontacthelical.sh: /content/drive/MyDrive/ColabNotebooks/Solubility-tool/SPIDER3-Single_np/script/spider3_pred.py: not found
Generating CCMPRED outputs...
  File "sources/CCMpred/a3m_to_ali.py", line 9
    print x.seq
          ^
SyntaxError: Missing parentheses in call to 'print'. Did you mean print(x.seq)?
sources/CCMpred

In [28]:
!ls /content/drive/MyDrive/ColabNotebooks/Solubility-tool/ncbi-blast-2.11.0+

bin  ChangeLog	doc  LICENSE  ncbi_package_info  README
